In [1]:
import os
import pandas as pd
import math
import tarfile
from six.moves import urllib
import hashlib
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TreebankWordTokenizer
from bs4 import BeautifulSoup
import xml.etree.ElementTree
import mailbox   
import lxml.html
import lxml.html.clean
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.stem import PorterStemmer
import email
from sklearn.pipeline import Pipeline
# Import the email modules we'll need
from email.parser import Parser
import mailparser as mp
from email.mime.text import MIMEText
import nltk
from nltk import pos_tag
import re
from collections import Counter



# path = spam_path
# i = 0
# for filename in os.listdir(path):
#     print('orig', filename)
#     os.rename(os.path.join(path,filename), os.path.join(path,'mail_'+str(i)))
#     print('changed', filename)
#     i = i +1
# print(i)




class preparing_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, x, y=None):
        return self
    def transform(self, x, y=None):
        #print('type of x is' , type(x))
        #x = x.lower()
        m_body, m_headers, sub = dividing_mail(x)
        if m_body != '':
            m_body = m_body.lower()
            #m_body = m_body.split('.').join("");
            urls = counting_urls(m_body)
            m_body = cleaning_htmlent(m_body)
            
            name_sender, dom_sender = x.from_[0][0], x.from_[0][1].split('@')[-1]
        
            sub_list = keep_only_important_words(sub)
            body_list = keep_only_important_words(m_body)
            sub_list = removing_alphas(sub_list)
            body_list = removing_alphas(body_list)
            
            #save words record?
            ps = PorterStemmer()
            s_c =   [ps.stem(w) for w in sub_list]
            b_c =   [ps.stem(w) for w in body_list]
            b_c.extend([urls, name_sender, dom_sender])
            
            
        else:
            print('returning nothing')
            s_c =['','','']
            b_c = ['','','']
        return s_c, b_c
        



            
def keep_only_important_words(data_txt):
        m_words =[]
        for word, pos in nltk.pos_tag(TreebankWordTokenizer().tokenize(data_txt)):
            #if pos not in ['IN', '.', 'CD', 'DT', 'RB', 'VBP', 'TO', 'PRP', 'C']: VB and VBP removed
            if pos in ['NN',  'JJ', 'VBN'] :
                m_words.append(word)
            else:
                pass
        return(m_words)

def cleaning_htmlent(text):
    if text != '':
        doc = lxml.html.fromstring(text)
        cleaner = lxml.html.clean.Cleaner(style=True)
        doc = cleaner.clean_html(doc)
        content = doc.text_content()
        return content
    else:
        return []

def counting_urls(text):
#     soup = BeautifulSoup(text, 'html.parser')
#     num = soup.find_all('a')
#     print('number is ', (num))

    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    num_urls = len(urls)
    return num_urls
  
def dividing_mail(mail):
        m_body = mail.body
        m_headers = mail.headers
        
        sub = mail.subject
        return m_body, m_headers, sub

def removing_alphas(doc):
    dellist = []
    for i in range(len(doc)):
        if not doc[i].isalpha():
            dellist.append(i)
       
    for index in sorted(dellist, reverse=True):
        del doc[index]
    return doc
    
    

def extracting_feats(body_list, sub_list, arr):
    i = 0
    fl = []
    for  l in list([body_list, sub_list]):
        flatten_list = [a for t in l for a in t]
        out = Counter(flatten_list)
        out = out.most_common(arr[i])
        i = i + 1 
        cols = [m[0] for m in out]
        fl.append(cols)
    return fl
      
    
    
    
sub_list = []
body_list = []




#Giving filepaths for the data
dat_path = "datasets/spamham"
spam_path = dat_path + '/easy_spam'
ham_path = dat_path + '/easy_ham'
len_spam = len(next(os.walk(spam_path))[2])
len_ham = len(next(os.walk(ham_path))[2])
ar_dict = [(spam_path, len_spam), (ham_path, len_ham)] 
sl = math.ceil(0.8 * len_spam)
hl = math.ceil(0.8 * len_ham)

y_train = [0] * sl + [1] * hl
n_list = []
    
for sp, num in ar_dict:
    tr_n = math.ceil(0.8 * int(num))
    print('trainining number: ', tr_n)
    
    for file_num in range(tr_n):
        #Reading an email and getting the content
        data_mail = sp + '/mail_' + str(file_num)
        #print(data_mail)
        
        
        mail = mp.parse_from_file(data_mail)
        obj = preparing_data()
        s, b = obj.fit_transform(mail)
        n_list.append(b[-3:])
        sub_list.append(s)
        body_list.append(b[:-3])
        
            

featurevectors = extracting_feats(body_list[:len_spam], sub_list[:len_spam], [200 ,50])
featurevectorh = extracting_feats(body_list[len_spam:], sub_list[len_spam:], [300 ,50])
#feat_vals = set(featurevectors) + set(featurevectorh)




trainining number:  400
returning nothing
returning nothing
trainining number:  2041


More than one match found for (?:with(?! cipher)\s+(?P<with>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+from|\s+by|\s+id|\s+for|\s+via|;)) in from unknown HELO ?192.168.0.100? salimma1@212.18.241.211 with plain by smtp.mail.vip.sc5.yahoo.com with SMTP; 10 Oct 2002 10:30:25 -0000
More than one match found for (?:with(?! cipher)\s+(?P<with>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+from|\s+by|\s+id|\s+for|\s+via|;)) in from p977.as2.cra.dublin.eircom.net HELO mfrenchw2k mfrench42@159.134.179.209 with login by smtp.mail.vip.sc5.yahoo.com with SMTP; 22 Aug 2002 22:02:25 -0000
More than one match found for (?:id\s+(?P<id>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+from|\s+by|\s+with(?! cipher)|\s+for|\s+via|;)) in from chassid 4.65.20.230 by out001.verizon.net InterMail vM.5.01.05.09 201-253-122-126-109-20020611 with ESMTP id <20021005011206.OGMC3265.out001.verizon.net@chassid> for <rpm-list@freshrpms.net>; Fri, 4 Oct 2002 20:12:06 -0500
More than one match fo

In [2]:
bodycols = set(featurevectors[0] + featurevectorh[0])
subcols = set(featurevectors[1] + featurevectorh[1])


sample_value = []


for num in range(hl + sl):
    for feature in bodycols:
        #print(feature)
        if feature in body_list[num]:
            sample_value.append(body_list[num].count(feature))
            
        else:
            sample_value.append(0)
                #print(sample_value)
    for feature in subcols:
        #print(len(sample_value))
        if feature in sub_list[num]:
            sample_value.append(sub_list[num].count(feature))
                #print(sample_value)
        else:
            sample_value.append(0)
            #print(sample_value)
    sample_value.extend(n_list[num])
            
   
print(len(sample_value))

1169239


In [11]:
data = np.reshape(sample_value, (2441, 479)) 
print(data.shape)


colnames =  list(bodycols) + list(subcols) + ['URL_num', 'SenderName', 'SenderDomain']

df = pd.DataFrame(data = data, columns = colnames)
print(df.loc[0])

(2441, 479)
issu                              0
requir                            0
version                           0
softwar                           0
thing                             0
paid                              0
directori                         0
request                           0
limit                             0
intellig                          0
base                              0
custom                            0
secur                             0
anyon                             0
next                              0
url                               0
design                            0
cash                              0
fork                              0
notic                             0
powel                             0
found                             0
mailto                            0
war                               0
avail                             1
onli                              0
deal                              0
info            

In [ ]:
d = [('a', 1), ('b', 4), ('e', 3)]
d[0] 
m_words =[]
data_txt = 'i am and happy your'
for word, pos in nltk.pos_tag(TreebankWordTokenizer().tokenize(data_txt)):
    if pos not in ['IN', '.', 'CD', 'DT', 'RB', 'VBP']:
        print (word, pos)

In [ ]:
fl = []
flatten_list = [a for t in sub_list for a in t]
out = Counter(flatten_list)
out = out.most_common(20)
cols = [m[0] for m in out]
fl.append(cols)
fl
      

In [ ]:
len(n_list)